In [131]:
import tensorflow as tf
import numpy as np
import pandas as pd
from keras.layers import Conv2D, Flatten, Dropout, MaxPool2D, BatchNormalization, Reshape, Dense
from keras import Sequential
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#### Loading Data

In [132]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

#### Normalizing Data

In [133]:
X_train = train.drop(['label'],axis=1)
X_train = X_train/255.0
X_train = X_train.values.reshape(-1, 28, 28, 1)

In [134]:
test = test/255.0
test = test.values.reshape(-1, 28, 28, 1)

#### Label Encoding
Since the labelling is from [0,9] and since the values are not ordinal, we one hot encode them

In [135]:
y_train = tf.keras.utils.to_categorical(train['label'], num_classes=10)

Loading MNIST Dataset to increase size of our training dataset

In [136]:
from tensorflow.keras.datasets import mnist
(X_train_MNIST, y_train_MNIST), (X_test_MNIST, y_test_MNIST) = mnist.load_data()

In [137]:
print(X_train_MNIST.shape)
print(y_train_MNIST.shape)

(60000, 28, 28)
(60000,)


In [138]:
train_mnist = np.concatenate((X_train_MNIST, X_test_MNIST))
test_mnist = np.concatenate((y_train_MNIST, y_test_MNIST))

In [139]:
print(train_mnist.shape)
print(test_mnist.shape)

(70000, 28, 28)
(70000,)


In [140]:
train_mnist = train_mnist.reshape(-1, 28, 28, 1)
train_mnist = train_mnist/255.0

In [141]:
test_mnist = tf.keras.utils.to_categorical(test_mnist, num_classes=10)

Concatenating both datasets

In [142]:
images = np.concatenate((X_train, train_mnist))
labels = np.concatenate((y_train, test_mnist))

In [143]:
print(images.shape)
print(labels.shape)

(112000, 28, 28, 1)
(112000, 10)


### Data Augmentation

Adds rotation, zoom, horizontal movement to the present images, creating an even larger dataset, prevents overfitting and is a very strong tool

In [144]:
from keras.preprocessing.image import ImageDataGenerator

In [145]:
datagen = ImageDataGenerator(rotation_range=20, width_shift_range=0.2, shear_range=15, zoom_range=0.1, validation_split=0.25)

In [146]:
train_dataset = datagen.flow(images, labels, batch_size=256, subset='training')

In [147]:
test_dataset = datagen.flow(images, labels, batch_size=64, subset='validation')

### CNN Model

In [148]:
model = Sequential()

In [149]:
model.add(Conv2D(filters=32, kernel_size=5, activation='relu', padding='same', input_shape=(28, 28, 1)))
model.add(MaxPool2D(pool_size=2))
model.add(Conv2D(filters=64, kernel_size=3, activation='relu', padding='same'))
model.add(Conv2D(filters=64, kernel_size=3, activation='relu', padding='same'))
model.add(MaxPool2D(pool_size=2))
model.add(Conv2D(filters=64, kernel_size=3, activation='relu', padding='same'))
model.add(Conv2D(filters=64, kernel_size=3, activation='relu', padding='same'))
model.add(MaxPool2D(pool_size=2))

In [150]:
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(256, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [151]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [154]:
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=0.000001)
checkpoint = ModelCheckpoint(filepath='model.hdf5', monitor='val_loss', save_best_only=True, save_weights_only=True)

In [155]:
model.fit_generator(train_dataset, epochs=70, validation_data=test_dataset, callbacks=[reduce_lr, checkpoint])

C:\Users\shvmt\AppData\Local\Temp/ipykernel_7612/4107944639.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_dataset, epochs=70, validation_data=test_dataset, callbacks=[reduce_lr, checkpoint])


Epoch 1/70
329/329 [==============================] - 71s 214ms/step - loss: 0.3964 - accuracy: 0.8696 - val_loss: 0.1014 - val_accuracy: 0.9688 - lr: 0.0010
Epoch 2/70
329/329 [==============================] - 70s 213ms/step - loss: 0.0955 - accuracy: 0.9717 - val_loss: 0.0631 - val_accuracy: 0.9804 - lr: 0.0010
Epoch 3/70
329/329 [==============================] - 73s 221ms/step - loss: 0.0687 - accuracy: 0.9797 - val_loss: 0.0572 - val_accuracy: 0.9828 - lr: 0.0010
Epoch 4/70
329/329 [==============================] - 73s 223ms/step - loss: 0.0580 - accuracy: 0.9835 - val_loss: 0.0539 - val_accuracy: 0.9835 - lr: 0.0010
Epoch 5/70
329/329 [==============================] - 70s 211ms/step - loss: 0.0509 - accuracy: 0.9850 - val_loss: 0.0365 - val_accuracy: 0.9893 - lr: 0.0010
Epoch 6/70
329/329 [==============================] - 72s 218ms/step - loss: 0.0451 - accuracy: 0.9869 - val_loss: 0.0390 - val_accuracy: 0.9888 - lr: 0.0010
Epoch 7/70
329/329 [==============================] 

329/329 [==============================] - 72s 220ms/step - loss: 0.0068 - accuracy: 0.9980 - val_loss: 0.0074 - val_accuracy: 0.9974 - lr: 1.0000e-04
Epoch 53/70
329/329 [==============================] - 72s 220ms/step - loss: 0.0070 - accuracy: 0.9979 - val_loss: 0.0079 - val_accuracy: 0.9977 - lr: 1.0000e-04
Epoch 54/70
329/329 [==============================] - 73s 221ms/step - loss: 0.0067 - accuracy: 0.9980 - val_loss: 0.0064 - val_accuracy: 0.9980 - lr: 1.0000e-04
Epoch 55/70
329/329 [==============================] - 73s 220ms/step - loss: 0.0073 - accuracy: 0.9980 - val_loss: 0.0065 - val_accuracy: 0.9978 - lr: 1.0000e-04
Epoch 56/70
329/329 [==============================] - 73s 223ms/step - loss: 0.0064 - accuracy: 0.9980 - val_loss: 0.0068 - val_accuracy: 0.9979 - lr: 1.0000e-04
Epoch 57/70
329/329 [==============================] - 72s 220ms/step - loss: 0.0062 - accuracy: 0.9981 - val_loss: 0.0053 - val_accuracy: 0.9983 - lr: 1.0000e-05
Epoch 58/70
329/329 [=============

In [158]:
submissions = pd.read_csv('sample_submission.csv')

#### Predicting Values

In [161]:
ans = tf.keras.backend.argmax(model.predict(test))

In [165]:
submission=pd.DataFrame({'ImageId': range(1, len(ans)+1), 'label': ans})
submission.to_csv('submitfinal.csv', index=False)